In [21]:
%pip install transformers peft accelerate bitsandbytes

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:
%pip install -U "transformers>=4.36.2" "peft>=0.5.0"


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
import accelerate
print(accelerate.__version__)


0.21.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# Load mô hình Vit5-base từ local + offload folder
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    "../vit5-base",
    device_map="auto",
    offload_folder="./offload"  # thư mục lưu tạm weight khi thiếu RAM
)

tokenizer = AutoTokenizer.from_pretrained("../vit5-base", use_fast=False)

# Load adapter QLoRA đã fine-tuned
model = PeftModel.from_pretrained(
    base_model,
    "../models/vit5-vietnews-qlora/checkpoint-13000",
    is_trainable=False
)

model.eval()
print("✅ Mô hình đã được load thành công!")


ImportError: cannot import name 'clear_device_cache' from 'accelerate.utils.memory' (C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\utils\memory.py)

In [ ]:
import torch


def summarize(text):
    prompt = f"Tóm tắt:\n{text.strip()}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.5, 
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)



In [ ]:
sample = """
Bức ảnh là một CV (Curriculum Vitae) của một người tên Lê Hoàng Vũ, sinh năm 2004. CV bao gồm các phần sau:

1. *Thông tin cá nhân:*
    - Tên: Lê Hoàng Vũ
    - Ngày sinh: 2004
    - Giới tính: Nam
    - Nghề nghiệp: GIA SƯ TOÁN, LẬP TRÌNH

2. *Kỹ năng:*
    - Tư duy, suy luận, làm sáng tỏ
    - Công Nghệ Thông Tin
    - Giao tiếp, truyền đạt kiến thức
    - Soạn thảo tài liệu bằng LaTec

3. *Học vấn:*
    - Trường THPT Chuyên Bảo Lộc
    - 2019 - 2022
    - 3 năm liên học sinh giỏi toàn diện
    - Đội tuyển Toán Quốc gia năm lớp 11, 12

4. *Chuyên Toán:*
    - 3 năm liên học sinh giỏi toàn diện
    - Đội tuyển Toán Quốc gia năm lớp 11, 12
    - Điểm trung bình 3 năm lần lượt là 9.1; 9.1 và 9.4
    - Trong đó điểm môn Toán lần lượt là 9.6; 9.9 và 9.6

5. *Giới thiệu:*
    - Từ lâu em đã có sự đam mê to lớn và tìm hiểu nghiên cứu lâu dài với Toán học. Từ đó mở ra một con đường mới với mảng Công nghệ Thông tin, Trí tuệ Nhân tạo.

6. *Kinh nghiệm làm việc:*
    - Giảng viên, trợ giảng
    - Đứng lớp giảng dạy, hỗ trợ các em học sinh khóa dưới trong đội tuyển Toán ở trường cấp 3

7. *Hoạt động:*
    - Liên hệ: 12/08/2004
    - Email: lehoangvm10@gmail.com
    - Địa chỉ: Tòa S10.05 Vinhomes GrandPark Origami, Q9, TPHCM
    - Điện thoại: 0786798668
    - LinkedIn: ProRain-HCMUS

8. *Điều quan tâm:*
    - Toán ứng dụng
    - Machine Learning (Học máy)
    - IoT (Internet of Things)

9. *Giải thưởng:*
    - Học sinh giỏi Toán cấp 3 tỉnh cấp 3
    - Kỳ thi Toán quốc tế FMC Bronze Medal 2022
    - HCB Kỳ thi Olympic truyền thống 30/4

"""


print("📄 Tóm tắt:", summarize(sample))


📄 Tóm tắt: Ngày 19/4, TAND tỉnh Vĩnh_Long mở phiên_toà sơ_thẩm xét_xử và tuyên_phạt Huỳnh_Công_Thành, tự Quốc 18 năm tù_giam về tội Vận_chuyển trái_phép chất ma_tuý, bị_cáo 18 tuổi.


In [ ]:
from typing import List
from sentence_transformers import SentenceTransformer, util

# ✅ Load mô hình embedding 1 lần (nếu chưa)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def summarize_best(text: str, num_trials: int = 5) -> str:
    """
    Sinh nhiều bản tóm tắt từ hàm `summarize()` rồi chọn bản tốt nhất 
    dựa trên cosine similarity giữa embedding của bản gốc và bản tóm tắt.

    Args:
        text (str): Đoạn văn cần tóm tắt.
        num_trials (int): Số lần gọi `summarize` để sinh kết quả.

    Returns:
        str: Bản tóm tắt được đánh giá là tốt nhất.
    """

    # ✅ Sinh nhiều bản tóm tắt
    summaries: List[str] = [summarize(text) for _ in range(num_trials)]

    # ✅ Tính embedding gốc
    text_embedding = embedder.encode(text, convert_to_tensor=True)

    # ✅ Chấm điểm từng tóm tắt
    scores = []
    for s in summaries:
        summary_embedding = embedder.encode(s, convert_to_tensor=True)
        similarity = util.cos_sim(text_embedding, summary_embedding).item()
        scores.append(similarity)

    # ✅ Chọn bản tóm tắt có điểm cao nhất
    best_index = scores.index(max(scores))
    return summaries[best_index]


Error importing huggingface_hub._snapshot_download: cannot import name 'GatedRepoError' from 'huggingface_hub.errors' (C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\errors.py)


ImportError: cannot import name 'GatedRepoError' from 'huggingface_hub.errors' (C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\errors.py)